In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import spacy
import re

In [ ]:
#you can replace the input data with your own corpus
df = pd.read_json("../input/news-data-assignment-2/relevant_articles.json")

In [5]:
df.head()

,_id,url,title,feed,type,pub,ret,lang,refs,sum,body,text,auth
0,197727,http://www.theguardian.com/housing-network/201...,Five steps to fixing the UK housing crisis in ...,guardian,politics,{'$date': '2016-01-01T09:12:27.000+0000'},{'$date': '2016-01-01T09:27:32.000+0000'},en,[http://www.theguardian.com/commentisfree/2015...,Headlines about the utter madness of our housi...,Perhaps in years to come 2015 will be remember...,Headlines about the utter madness of our housi...,NaN
1,197728,http://www.theguardian.com/commentisfree/2016/...,That’s enough assessments of 2015. It’s time t...,guardian,politics,{'$date': '2016-01-01T09:37:06.000+0000'},{'$date': '2016-01-01T09:47:42.000+0000'},en,"[http://www.theguardian.com/business/davos, ht...",From Strictly Come Dancing featuring Vladimir ...,• Jeremy Corbyn turns down invitation to atten...,From Strictly Come Dancing featuring Vladimir ...,NaN
2,197995,http://www.theguardian.com/politics/2016/jan/0...,The paperback memoirs of Miss Rice-Davies : ar...,guardian,politics,{'$date': '2016-01-01T05:00:22.000+0000'},{'$date': '2016-01-01T22:59:09.000+0000'},en,[http://www.theguardian.com/politics/2014/dec/...,1 January 1964 : The Mandy Report turns out to...,"Miss Mandy Rice-Davies, one of 1963’s better k...",1 January 1964 : The Mandy Report turns out to...,NaN
3,198038,http://www.bbc.co.uk/news/uk-scotland-35210821...,'Very drunk' patient numbers revealed,bbc,politics,{'$date': '2016-01-02T00:42:46.000+0000'},{'$date': '2016-01-02T00:45:47.000+0000'},en,[http://www.bbc.co.uk/news/uk-scotland-35097230],Ambulances attend more than 60 incidents on av...,"Paramedics treated about 12,000 people who wer...",Ambulances attend more than 60 incidents on av...,NaN
4,198046,http://www.bbc.co.uk/news/uk-35212696#sa-ns_mc...,Littering fines set to rise to £150,bbc,politics,{'$date': '2016-01-02T04:28:41.000+0000'},{'$date': '2016-01-02T04:47:52.000+0000'},en,[http://www.telegraph.co.uk/news/politics/1206...,Penalties for people who drop litter could rea...,"According to the Daily Telegraph, the minimum ...",Penalties for people who drop litter could rea...,NaN


In [6]:
df_short = df[:20000]
df_short.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   _id     20000 non-null  int64 
 1   url     20000 non-null  object
 2   title   20000 non-null  object
 3   feed    20000 non-null  object
 4   type    20000 non-null  object
 5   pub     20000 non-null  object
 6   ret     20000 non-null  object
 7   lang    20000 non-null  object
 8   refs    15247 non-null  object
 9   sum     10443 non-null  object
 10  body    10443 non-null  object
 11  text    20000 non-null  object
 12  auth    13326 non-null  object
dtypes: int64(1), object(12)
memory usage: 2.0+ MB


#### Single function to deal with new dataset of raw text

In [1]:
#download the spacy pre-trained model that we will use to process our corpus in iteration.
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 782.7/782.7 MB 1.0 MB/s eta 0:00:0000:0100:03
  Preparing metadata (setup.py) ... done
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [3]:
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer
import re
import time

In [4]:
#load the json file
df = pd.read_json("../input/news-data-assignment-2/relevant_articles.json")

#set up spacy nlp model
nlp = spacy.load("en_core_web_lg")


In [15]:
def clean_corpus(raw_text):
    assert isinstance(raw_text,str)
    
    #precleaning with rege
    cleaned_text = re.sub(r"http\S+","", raw_text)  #remove URL
    cleaned_text = re.sub(r"[^a-zA-Z]"," ", cleaned_text)   #remove non-numeric
    cleaned_text = re.sub(r"\n|\\"," ", cleaned_text)   #specific remove for my dataset
    
    #lower case
    cleaned_text = str(cleaned_text).lower()
       
    #spacy tokenization
    tokenizer = Tokenizer(nlp.vocab)
    spacy_token_doc = tokenizer(cleaned_text)
    token_list = []
    for token in spacy_token_doc:
        token_list.append(token.text)
    cleaned_text = token_list
    
    #first round of removing stopwords by spacy vocabulary
    stopwords = nlp.Defaults.stop_words
    filtered_tokens = []    
    for word in cleaned_text:
        if nlp.vocab[str(word)].is_stop == False:
            filtered_tokens.append(word)
    cleaned_text = filtered_tokens
    
    #First convertion of list into string, since the spacy pipeline cannot deal with
    cleaned_text = ' '.join(cleaned_text)
    
    #Lemmatization with spacy model, super long time!
    doc = nlp(cleaned_text)
    cleaned_text = [token.lemma_ for token in doc]
    
    #second round of removing stopwords by spacy vocabulary
    filtered_lemmas = []    
    for word in cleaned_text:
        if nlp.vocab[str(word)].is_stop == False:
            filtered_lemmas.append(word)
    cleaned_text =filtered_lemmas
    
    #Remove the words having length <= 2    
    cleaned_text = [i for i in cleaned_text if len(i) > 2]
    
    #remove all the blank space elements from the lemma list    
    no_blanke_lemmas = []
    for lemma in cleaned_text:
        if lemma.strip():
            no_blanke_lemmas.append(lemma)
    cleaned_text = no_blanke_lemmas
    
    #second convertion of list into string    
    cleaned_text = ' '.join(cleaned_text)
                            
    return cleaned_text
    

In [16]:
# df_short = df[:20]
start = time.time()
df_short["clean_text"] = df_short["text"].apply(clean_corpus)
print("This process took: {} seconds.".format(time.time() - start))

This process took: 1.4619503021240234 seconds.


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [17]:
for i in range(5):
    print("New Text: {}".format(df_short["clean_text"].iloc[i]))

New Text: headline utter madness housing market dominate time new year resolution year come remember year housing crisis mainstream fellow housing economic journalist wail like cassandra year year rare flip newspaper headline address utter madness housing market right buy extended house price south east london rocket footballer unlikely champion homeless campaign eviction rate soar new year opportunity fresh start new year resolutions britain housing crisis relate housing month grow crisis market capricious design provide household struggle provide state function safety net overlook housing accept municipal housing run council housing association necessary prudent investment prevent far costly homelessness government focus private home ownership benefit comfortable country affordable home rent buy unavailable housing market profit barely regulate continue reap reward wholesale flog council housing wonder people bedroom tax day christmas conservative sneak report impact penalty people p

#### Save the dataframe to a csv file

In [ ]:
#if necessary
df_short.to_csv(file_name, sep='\t')